In [128]:
import pandas as pd
import geopandas

In [129]:
poi = geopandas.read_file("data/points-of-interest.geojson")
samm = geopandas.read_file("data/sammelstellen.geojson")
robidog = geopandas.read_file("data/hundekotbehalter.geojson")
mobility = geopandas.read_file('data/mobility-stationen-und-fahrzeuge-schweiz.geojson')
begegnung = geopandas.read_file('data/begegnungszonen.geojson')
tempo30 = geopandas.read_file('data/tempo-30-zonen.geojson')
veloplaetze = geopandas.read_file('data/veloabstellplatze.geojson')
parkplaetze = geopandas.read_file('data/parkplatze-und-parkhauser-stadt-stgallen.geojson')
beleuchtung = geopandas.read_file('data/offentliche-beleuchtung-stadt-stgallen.geojson')
eladestation = geopandas.read_file('data/ladestationen-fur-elektroautos-im-kanton-stgallen.geojson')
ovhaltestelle = geopandas.read_file('data/haltestelle-didok.geojson')
eventplaetze = geopandas.read_file('data/eventplatze-offentlicher-raum-stadt-stgallen.geojson')

In [201]:
datasets = {
    'Sammelstelle': samm,
    'Robidog': robidog,
    # 'Mobility': mobility,
    'Begegnung': begegnung,
    'Tempo30': tempo30,
    'VeloPlaetze': veloplaetze,
    'Parkplaetze': parkplaetze,
    # 'Beleuchtung': beleuchtung,
    # 'E-Ladestation': eladestation,
    'OV-Haltestelle': ovhaltestelle,
    'EventPlaetze': eventplaetze
}

In [179]:
merged = geopandas.read_file('data/merged.csv')
data = geopandas.GeoDataFrame(merged, geometry=geopandas.points_from_xy(merged.longitude, merged.latitude))
data = data.set_crs('EPSG:4326', allow_override=True) # coordinates
print('CRS added:',data.crs is not None)
print('Columns before:', len(data.columns))
print('Rows before:',len(data.index))

CRS added: True
Columns before: 19
Rows before: 44945


In [180]:
def join_nearest(left_set, right_set, title, max_dist=1000, plot=True):
    col_name = f'dist_to_{title.lower()}'
    if col_name in left_set.columns:
        print(f'{title} exists already, skipping...')
        return left_set
    else:
        left_meters = left_set.to_crs('EPSG:3857') # meters
        right_meters = right_set.to_crs('EPSG:3857') # meters
        res = left_meters.sjoin_nearest(right=right_meters, how='left', max_distance=max_dist, distance_col=col_name, lsuffix=title.lower())
        # res = res.loc[:, res.columns != 'index_right']
        res = res.loc[:, res.columns == col_name]
        left_set[col_name] = res
        if plot:
            res.plot(col_name, legend=True)
        print(f'{title} added')
        return left_set

In [202]:
for x, d in enumerate(datasets):
    print('dataset:', f'{x+1}/{len(datasets)}')
    data = join_nearest(data, datasets[d], d, max_dist=10000, plot=False)
print('Columns after:', len(data.columns))
print('Rows after:',len(data.index))

dataset: 1/9
Sammelstelle exists already, skipping...
dataset: 2/9
Robidog exists already, skipping...
dataset: 3/9


/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  super().__setitem__(key, value)


ValueError: cannot reindex on an axis with duplicate labels

In [167]:
data


,field_1,coord_x,coord_y,city,canton,year,street_name,street_number,multiple_floors,num_rooms,...,latitude,price,street_safety_score,geometry,dist_to_sammelstelle,dist_to_robidog,dist_to_begegnung,dist_to_tempo30,dist_to_veloplaetze,dist_to_parkplaetze
0,0,2747136.1,1256043.3,St. Gallen,SG,1965.0,Unterer Hompeliweg,4,0.0,2.0,...,47.43872666567338,,26.708782183427797,POINT (9.38934 47.43873),644.074956,89.988499,1952.029222,0.000000,298.850558,54.173893
1,1,2745364.3,1254156.2,St. Gallen,SG,1918.0,Rosenbergstrasse,91,0.0,3.0,...,47.422152607247895,,8.69700949838709,POINT (9.36524 47.42215),658.903781,229.912435,225.241834,32.533245,47.849680,138.720386
2,2,2748011.7,1255777.0,St. Gallen,SG,1993.0,Lindenstrasse,126,0.0,4.0,...,47.436135950539,,17.934163974995577,POINT (9.40085 47.43614),814.606751,321.852793,2235.347240,0.000000,359.827386,20.763415
3,3,2746893.3,1254690.5,St. Gallen,SG,1919.0,Konkordiastrasse,34,0.0,2.0,...,47.426617352823904,,38.06644858574981,POINT (9.38567 47.42662),418.028153,180.784431,8.787023,0.000000,131.839969,17.072152
4,4,2745703.6,1252892.3,St. Gallen,SG,2009.0,Wattstrasse,13,0.0,4.0,...,47.41071327626508,,32.640382373512246,POINT (9.36933 47.41071),355.818808,116.108031,609.836104,0.000000,304.286414,317.911586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44940,44940,2746823.3,1254909.0,St. Gallen,SG,1950.0,Volksbadstrasse,8,0.0,2.0,...,47.42859759005008,,29.812277998256118,POINT (9.38482 47.42860),485.880798,135.719144,224.694868,0.000000,51.285126,7.240906
44941,44941,2745070.9,1254587.6,St. Gallen,SG,1964.0,Schorenstrasse,50,0.0,2.0,...,47.426096245766495,,15.999024929727952,POINT (9.36150 47.42610),125.284982,459.639440,782.964975,0.000000,93.306868,40.428942
44942,44942,2745008.3,1253419.7,St. Gallen,SG,1918.0,Vonwilstrasse,5,0.0,4.0,...,47.41560887577516,,36.22394403284855,POINT (9.36029 47.41561),504.015573,177.847360,400.891676,64.358629,242.225028,18.619930
44943,44943,2745799.0,1253988.6,St. Gallen,SG,1953.0,Teufener Strasse,12,0.0,3.0,...,47.42054951709161,,27.81168933207453,POINT (9.37095 47.42055),144.780642,247.463608,70.077493,71.013461,16.002250,13.642580


In [203]:
data.to_csv('merged_new_cols.csv')